In [1]:
import spacy
import csv
import pandas as pd
import math
from spacy.tokens import DocBin
from spacy import displacy

In [2]:
def find_largest_common_substring(str1, str2):
    m = len(str1)
    n = len(str2)

    # Create a table to store the lengths of the common substrings
    table = [[0] * (n + 1) for _ in range(m + 1)]

    # Variables to store the length and ending position of the longest common substring
    max_length = 0
    end_position = 0

    # Iterate through each character of str1 and str2
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                table[i][j] = table[i - 1][j - 1] + 1
                if table[i][j] > max_length:
                    max_length = table[i][j]
                    end_position = i

    # Extract the longest common substring
    longest_common_substring = str1[end_position - max_length: end_position]

    return longest_common_substring.strip()

In [3]:
df = pd.read_csv('2complete.csv')
# specs_df = pd.read_csv('table_data_refrigerator.csv')
df = df.sample(frac = 1)
# train_df = df.sample(frac=0.6, random_state=200)
# test_df = df.drop(train_df.index)
# test_df
df.head()

,ID,Title,Brand,Model
3776,3777,"Yakumo 410 comes with a 1/2.5"" (~ 5.75 x 4.3...",Yakumo,Mega Image 410
956,957,"Nikon L16 comes with a 1/2.5"" (~ 5.75 x 4.32...",Nikon,Coolpix L16
576,577,"Fujifilm J25 comes with a 1/2.3"" (~ 6.16 x 4...",Fujifilm,FinePix J25
567,568,"Fujifilm S1000fd comes with a 1/2.3"" (~ 6.16...",Fujifilm,FinePix S1000fd
1805,1806,"Ricoh GX100 comes with a 1/1.75"" (~ 7.31 x 5...",Ricoh,Caplio GX100


In [4]:
data=[]
for ind in df.index:
    id = df['ID'][ind]
    title = df['Title'][ind]
    brand = df['Brand'][ind]
    model = df['Model'][ind]
    brand_start = title.find(brand)
    x = find_largest_common_substring(model,title)
    y= title.find(x)
    data.append({'ID':id, 'Title': title, 'Brand': brand, 'Model': model, 'brand_start': brand_start, 'brand_end': brand_start + len(brand), 'model_start': y, 'model_end': y + len(x)})
train = data[:math.floor(len(data)*0.6)]
test = data[math.floor(len(data)*0.6):]

In [5]:
# train_df = df.sample(frac=0.6, random_state=200)
# test_df = df.drop(train_df.index)

In [6]:
training_data = {'classes':['BRAND', 'MODEL'], 'annotations':[]}
for ex in train:
    temp_dict = {}
    temp_dict['text'] = ex['Title']
    temp_dict['entities'] = []
    temp_dict['entities'].append((ex['brand_start'], ex['brand_end'], 'BRAND'))
    temp_dict['entities'].append((ex['model_start'], ex['model_end'], 'MODEL'))
    training_data['annotations'].append(temp_dict)
training_data['annotations'][0]

{'text': 'Yakumo  410 comes with a  1/2.5" (~ 5.75 x 4.32 mm) CCD sensor, which has a diagonal of  7.19 mm (0.28") and a surface area of 24.84 mm².',
 'entities': [(0, 6, 'BRAND'), (8, 11, 'MODEL')]}

In [7]:
nlp = spacy.blank('en')
doc_bin = DocBin()

for training_example  in (training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [21]:
nlp_ner = spacy.load("model-best")
doc = nlp_ner('Along with a 2/3"" (~ 8.8 x 6.6 mm) CCD sensor,the Nikon E3s which has a diagonal of 11.00 mm.')
for ent in doc.ents:
    print(ent.text, ent.label_)
colors = {"MODEL": "#F67DE3", "BRAND": "#7DF6D9"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Along BRAND
Nikon BRAND
E3s which has a diagonal of 11.00 mm. MODEL
